# Using MACD and RSI together

## MACD and RSI - Separate Strategies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import brute
plt.style.use("seaborn")

In [ ]:
pd.read_csv("forex_pairs.csv")

In [ ]:
symbol = "EURUSD=X"

__MACD__

In [ ]:
from MACDBacktester import MACDBacktester as MACD

In [ ]:
ptc = 0.00007

In [ ]:
tester = MACD(symbol, EMA_S = 12, EMA_L = 26, signal_mw = 9,
              start = "2004-01-01", end = "2020-06-30", tc = ptc)

In [ ]:
tester.test_strategy()

In [ ]:
tester.optimize_parameters((5, 20, 1), (21, 50, 1), (5, 20, 1))

In [ ]:
tester.plot_results()

In [ ]:
macd = tester.results
macd

__RSI__

In [ ]:
from RSIBacktester import RSIBacktester as RSI

In [ ]:
tester = RSI(symbol, periods = 20, rsi_upper = 70, rsi_lower = 30,
              start = "2004-01-01", end = "2020-06-30", tc = ptc)

In [ ]:
tester.test_strategy()

In [ ]:
tester.optimize_parameters((5, 20, 1), (65, 80, 1), (20, 35, 1)) # higher volatility

In [ ]:
tester.plot_results()

In [ ]:
rsi = tester.results
rsi

## Combination

In [ ]:
comb = macd.loc[:, ["returns", "position"]].copy()

In [ ]:
comb

In [ ]:
comb.rename(columns = {"position":"position_MACD"}, inplace = True)

In [ ]:
comb["position_RSI"] = rsi.position.astype("int")

In [ ]:
comb

In [ ]:
comb["position_comb"] = np.where(comb.position_MACD == comb.position_RSI, comb.position_MACD, 0)

In [ ]:
comb.position_comb.value_counts()

In [ ]:
comb.position_comb.plot(figsize = (12, 8))
plt.show()

In [ ]:
comb["strategy"] = comb["position_comb"].shift(1) * comb["returns"]

In [ ]:
comb

In [ ]:
comb.dropna(inplace=True)

In [ ]:
comb["trades"] = comb.position_comb.diff().fillna(0).abs()

In [ ]:
comb.strategy = comb.strategy - comb.trades * ptc

In [ ]:
comb["creturns"] = comb["returns"].cumsum().apply(np.exp)
comb["cstrategy"] = comb["strategy"].cumsum().apply(np.exp)

In [ ]:
comb[["creturns", "cstrategy"]].plot(figsize = (12, 8), fontsize = 12)
plt.legend(fontsize = 12)
plt.show()